# Data processing

In [1]:
import pandas as pd
import numpy as np
import torch
from torch import nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset, WeightedRandomSampler
from biosppy.signals import ecg
import matplotlib.pyplot as plt
import os
import pickle
from tqdm import tqdm
import math
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, f1_score

In [2]:
data = pd.read_csv('original_data/train.csv', index_col='id')
train_y = data['y']
labels = train_y.to_numpy()

In [3]:
# See if there is any difference between using the filtered heartbeats or not
with open('data/heartbeat_templates_ecg.pkl', 'rb') as f:
    heartbeats = pickle.load(f)
    
# with open('data/heartbeat_filtered_ecg.pkl', 'rb') as f:
#     heartbeats = pickle.load(f)

In [4]:
np.random.seed(32)

def split_indices(n, val_pct):
    n_val = int(val_pct*n)
    idxs = np.random.permutation(n)
    return idxs[n_val:], idxs[:n_val]

In [5]:
class SignalDataset(Dataset):
    def __init__(self, signals, lengths, labels):
        self.signals = signals
        self.labels = labels
        self.lengths = lengths

    def __len__(self):
        return len(self.signals)

    def __getitem__(self, idx):
        return self.signals[idx], self.lengths[idx], self.labels[idx]

In [6]:
## Be sure that all the heartbeats have the same length

# Longest seq of beats
# max_length_heartbeats = max([len(i) for i in heartbeats])
max_length_heartbeats = 202 # Theres a long seq in test, maybe I can drop it...
beat_length = 180 # All heartbeats have the same length

# Padded heartbeats
padded_heartbeats = []
lengths = []

# Normalize before padding
for i,heartbeat in enumerate(heartbeats):
    heartbeats[i] = (heartbeat - heartbeat.mean(axis=-1, keepdims=True)) / heartbeat.std(axis=-1, keepdims=True)
    
for heartbeat in heartbeats:
    length = len(heartbeat)
    lengths.append(length)
    pad = np.zeros((max_length_heartbeats-length, 180))
    padded_heartbeats.append(np.concatenate((heartbeat, pad),axis=0))

In [7]:
# Transform heartbeats to tensor
padded_heartbeats = np.array(padded_heartbeats)
padded_heartbeats = torch.tensor(padded_heartbeats, dtype=torch.float32)
labels_heartbeats = torch.tensor(labels)
lengths_heartbeats = torch.tensor(lengths)

# Normalize data per beat
# padded_heartbeats = (padded_heartbeats - padded_heartbeats.mean(axis=-1, keepdims=True)) / padded_heartbeats.std(axis=-1, keepdims=True)

# Train val split
train_idxs, val_idxs = split_indices(len(padded_heartbeats), 0.2)
train_x = padded_heartbeats[train_idxs]
val_y = labels_heartbeats[val_idxs]
train_y = labels_heartbeats[train_idxs]
val_x = padded_heartbeats[val_idxs]
train_lengths = lengths_heartbeats[train_idxs]
val_lengths = lengths_heartbeats[val_idxs]

In [8]:
# Let's make a dataset for all the individual beats - 1D conv

labels_beats = []
beats = []

for i, beat in enumerate(heartbeats):
    samples = beat.shape[0]
    beats.append(torch.tensor(beat, dtype=torch.float32))
    labels_beats.append(torch.full((samples,), labels[i]))
    
beats = torch.cat(beats, dim=0)
labels_beats = torch.cat(labels_beats)

In [9]:
# Train - val split for individual beats

train_idxs_beats, val_idxs_beats = split_indices(len(beats), 0.2)
train_x_beats = beats[train_idxs_beats]
val_y_beats = labels_beats[val_idxs_beats]
train_y_beats = labels_beats[train_idxs_beats]
val_x_beats = beats[val_idxs_beats]

In [10]:
# For the moment training without a scaler but try also with scaler later.
# scaler = StandardScaler()
# train_x = scaler.fit_transform(train_x)
# val_x = scaler.transform(val_x)

### Create dataset

In [11]:
### Signal dataset
class SignalDataset(Dataset):
    def __init__(self, signals, lengths, labels):
        self.signals = signals
        self.labels = labels
        self.lengths = lengths

    def __len__(self):
        return len(self.signals)

    def __getitem__(self, idx):
        return self.signals[idx], self.lengths[idx], self.labels[idx]

In [12]:
train_dataset = SignalDataset(train_x, train_lengths, train_y)
val_dataset = SignalDataset(val_x, val_lengths, val_y)

In [13]:
# Theres a big class invariance in our dataset so let's try to fight it with class weights

class_counts = torch.tensor([3030, 443, 1474, 170], dtype=torch.float32)
class_weights = 1.0/class_counts
class_weights = class_weights/class_weights.sum()

In [16]:
# Oversample classes with lower appearance
sample_weights = [1.0 / class_counts[label] for label in train_y]
sample_weights = torch.tensor(sample_weights, dtype=torch.float32)
sampler = WeightedRandomSampler(weights=sample_weights, num_samples=len(sample_weights), replacement=True)

In [17]:
BATCH_SIZE = 128 #64

def collate_fn(batch):
    signals, lengths, labels = zip(*batch)
    signals = torch.stack(signals)
    lengths = torch.stack(lengths)
    labels = torch.stack(labels)
    signals = signals.to(device)
    lengths = lengths.to(device)
    attention_mask = torch.arange(signals.size(1), device=device).unsqueeze(0) >= lengths.unsqueeze(1)
    return signals, attention_mask, labels

train_dataloader = DataLoader(train_dataset, batch_size=BATCH_SIZE, collate_fn=collate_fn, sampler=sampler)
val_dataloader = DataLoader(val_dataset, batch_size=BATCH_SIZE, collate_fn=collate_fn)

# Transformer for signal beats

In [18]:
class PositionalEncoding(nn.Module):
    def __init__(self, d_model:int, seq_len:int, dropout:float):
        super(PositionalEncoding, self).__init__()
        self.d_model = d_model
        self.seq_len = seq_len
        self.dropout = nn.Dropout(dropout)
        
        pe = torch.zeros(seq_len, d_model) #(seq_len, d_model)
        
        position = torch.arange(0, seq_len, dtype=torch.float).unsqueeze(1) #(seq_len, 1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float()*(-math.log(10000.0)/d_model))
        pe[:,0::2] = torch.sin(position*div_term)
        pe[:,1::2] = torch.cos(position*div_term)
        
        pe = pe.unsqueeze(0) #(1, seq_len, d_model)
        
        self.register_buffer('pe', pe)
        
    def forward(self, x):
        x = x + (self.pe[:, :x.shape[1], :]).requires_grad_(False)
        return self.dropout(x)

In [19]:
class LayerNorm(nn.Module):
    def __init__(self, eps:float = 10**-6):
        super(LayerNorm, self).__init__()
        self.eps = eps
        self.alpha = nn.Parameter(torch.ones(1))
        self.bias = nn.Parameter(torch.zeros(1))
        
    def forward(self,x):
        mean = x.mean(dim=-1, keepdim=True) # We use the last dimension
        std = x.std(dim=-1, keepdim=True)
        return self.alpha*(x-mean)/(std+self.eps)+self.bias

In [20]:
class FeedFwd(nn.Module):
    def __init__(self, d_model:int, d_ff:int, dropout:float):
        super(FeedFwd, self).__init__()
        self.linear1 = nn.Linear(d_model, d_ff)
        self.dropout = nn.Dropout(dropout)
        self.linear2 = nn.Linear(d_ff, d_model)
        self.relu = nn.ReLU()
        
    def forward(self, x):
        # (b_size, seq_len, d_model) -> (b_size, seq_len, dff) -> (b_size, seq_len, d_model)
        x = self.relu(self.linear1(x))
        x = self.dropout(self.linear2(x))
        return x
        #return self.linear2(self.dropout(self.linear1(x)))

In [21]:
class MultiheadAttention(nn.Module):
    def __init__(self, d_model:int, h:int, dropout:float): # h -> num heads
        super(MultiheadAttention, self).__init__()
        self.d_model = d_model
        self.h = h
        assert d_model % h == 0, "d_model is not divisible by h"
        
        self.d_k = d_model // h # dim of each head
        
        self.w_q = nn.Linear(d_model, d_model)
        self.w_k = nn.Linear(d_model, d_model)
        self.w_v = nn.Linear(d_model, d_model)
        
        self.w_o = nn.Linear(d_model, d_model)
        self.dropout = nn.Dropout(dropout)
        
    @staticmethod
    def attention(query, key, value, mask, dropout):
        d_k = query.shape[-1]
        attention_scores = (query @ key.transpose(-2, -1)) / math.sqrt(d_k)
        if mask != None:
            mask = mask.unsqueeze(1).unsqueeze(2) # (b_size, 1, 1, seq_len) -> make this to match the attention_Score size
            #attention_scores.masked_fill_(mask == 0, -1e9)
            attention_scores.masked_fill_(mask, -1e9)
        attention_scores = attention_scores.softmax(dim=-1) # (b_size, h, seq_len, seq_len)
        if dropout != None:
            attention_scores = dropout(attention_scores)
            
        return (attention_scores@value), attention_scores
        
    def forward(self, q, k, v, mask):
        query = self.w_q(q) # (b_size, seq_len, d_model) -> (b_size, seq_len, d_model)
        key = self.w_k(k)
        val = self.w_v(v)
        
        query = query.view(query.shape[0], query.shape[1], self.h, self.d_k).transpose(1,2) # Divide to heads
        key = key.view(key.shape[0], key.shape[1], self.h, self.d_k).transpose(1,2)
        val = val.view(val.shape[0], val.shape[1], self.h, self.d_k).transpose(1,2)
        # Outpus size at this point (b_size, h, seq_len, d_k)
        
        # Apply mask
        x, self.attention_scores = MultiheadAttention.attention(query, key, val, mask, self.dropout)
        
        x = x.transpose(1,2).contiguous().view(x.shape[0], -1, self.h*self.d_k) # Go back to (b_size, seq_len, h, d_k) and then (b_size, seq_len, d_model)
        
        return self.w_o(x) # (b_size, seq_len, d_model)
    

In [22]:
class ResidualConnection(nn.Module):
    def __init__(self, dropout:float):
        super(ResidualConnection, self).__init__()
        self.dropout = nn.Dropout(dropout)
        self.norm = LayerNorm()
        
    def forward(self, x, sublayer):
        return x + self.dropout(sublayer(self.norm(x)))

In [23]:
class EncoderBlock(nn.Module):
    def __init__(self, self_attention, feed_forward, dropout):
        super(EncoderBlock, self).__init__()
        self.self_attention = self_attention
        self.feed_forward = feed_forward
        self.residual_connections = nn.ModuleList([ResidualConnection(dropout) for _ in range(2)])

    
    def forward(self, x, src_mask):
        x = self.residual_connections[0](x, lambda x: self.self_attention(x, x, x, src_mask))
        x = self.residual_connections[1](x, self.feed_forward)
        return x
        
        

In [24]:
class Encoder(nn.Module):
    def __init__(self, layers):
        super(Encoder, self).__init__()
        self.layers = layers
        self.norm = LayerNorm()
        
    def forward(self, x, mask):
        for layer in self.layers:
            x = layer(x, mask)
        return self.norm(x)

In [25]:
class Transformer(nn.Module):
    def __init__(self, encoder, pos_enc, src_size, d_model, output_dim, hidden=128):
        super(Transformer, self).__init__()
        self.encoder = encoder
        self.pos_enc = pos_enc
        self.input_proj = nn.Linear(src_size, d_model)
        self.hidden = nn.Linear(d_model, hidden)
        self.out_proj = nn.Linear(hidden, output_dim)
        self.act = nn.ReLU()
        
    def encode(self, src, src_mask):
        src = self.input_proj(src)
        src = self.pos_enc(src)
        encoder_output = self.encoder(src, src_mask)
        # MLP
        pre_out = encoder_output.mean(dim=1) #Avg pooling over seq length -> really helpful?
        # Attention pooling?
#         attention_weights = nn.Softmax(dim=1)(self.attention(encoder_output))
#         pre_out = torch.sum(encoder_output * attention_weights.unsqueeze(-1), dim=1)
        hidden_out = self.act(self.hidden(pre_out))
        logits = self.out_proj(hidden_out)
        return logits
        

In [26]:
def build_transformer(src_size, src_seq_len, d_model=512, N=6, h=8, dropout=0.1, d_ff=2048, output_dim=4):
    # Positional encoding
    pe = PositionalEncoding(d_model, src_seq_len, dropout)
    # Encoder
    encoder_blocks = []
    for _ in range(N):
        encoder_self_attention = MultiheadAttention(d_model, h, dropout)
        feed_fwd = FeedFwd(d_model, d_ff, dropout)
        encoder_block = EncoderBlock(encoder_self_attention, feed_fwd, dropout)
        encoder_blocks.append(encoder_block)
        
    encoder = Encoder(nn.ModuleList(encoder_blocks))
    
    transformer = Transformer(encoder, pe, src_size, d_model, output_dim)
    
    for p in transformer.parameters():
        if p.dim()>1:
            nn.init.xavier_uniform_(p)
            
    return transformer

In [27]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

d_model = 512
nhead = 8
num_encoder_layers=4
num_decoder_layers = 0
dim_feedforward = 2048
dropout = 0.1
output_dim = 4
LR = 1e-4
EPOCHS = 50

# src_seq_len=max_length_heartbeats
model = build_transformer(src_size=180, src_seq_len=max_length_heartbeats, d_model=d_model, N=num_encoder_layers, h=nhead, dropout=dropout, d_ff=dim_feedforward, output_dim=4)
model = model.to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=LR, weight_decay=1e-4)
loss_fn = nn.CrossEntropyLoss(weight=class_weights.to(device))
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=5)

train_losses = []
val_losses = []

for epoch in tqdm(range(EPOCHS)):
    model.train()
    train_loss=0
    all_preds = []
    all_labels = []
    for batch in train_dataloader:
        inputs, attention_mask, targets = batch
        inputs = inputs.to(device)
        attention_mask = attention_mask.to(device)
        targets = targets.to(device)
        optimizer.zero_grad()
        # Debug
#         inputs = torch.ones((1, 2, 180), device=device)
#         mask = torch.tensor([[False, True]], device=device)
        outputs = model.encode(inputs, attention_mask)
        pred = torch.argmax(outputs, dim=1)
        loss = loss_fn(outputs, targets)
        loss.backward()
        train_loss += loss.item()
        optimizer.step()
        
        all_labels.extend(targets.detach().cpu().tolist())
        all_preds.extend(pred.detach().cpu().tolist())
        
    val_loss = 0
    all_preds_val = []
    all_labels_val = []
    with torch.no_grad():
        model.eval()
        for batch in val_dataloader:
            inputs, attention_mask, targets = batch
            inputs = inputs.to(device)
            attention_mask = attention_mask.to(device)
            targets = targets.to(device)
            outputs = model.encode(inputs, attention_mask)
            pred = torch.argmax(outputs, dim=1)
            loss = loss_fn(outputs, targets)
            val_loss += loss.item()
            
            all_labels_val.extend(targets.detach().cpu().tolist())
            all_preds_val.extend(pred.detach().cpu().tolist())
            
    avg_train_loss = train_loss/len(train_dataloader)
    avg_val_loss = val_loss/len(val_dataloader)
    scheduler.step(avg_val_loss)
    train_acc = accuracy_score(all_labels, all_preds)
    val_acc = accuracy_score(all_labels_val, all_preds_val)
    f1 = f1_score(all_labels_val, all_preds_val, average='micro')
    print(f"Epoch [{epoch+1}/{EPOCHS}] | Train Loss: {avg_train_loss:.4f} | Validation Loss: {avg_val_loss:.4f} | Train acc: {train_acc:.4f} | Val acc: {val_acc:.4f} | f1 score: {f1:.4f}")
    train_losses.append(avg_train_loss)
    val_losses.append(avg_val_loss)
    
    # Check for vanishing gradient:
#     with torch.no_grad():
#         for name, param in model.named_parameters():
#             if param.grad is not None:
#                 print(f"{name} grad norm: {param.grad.norm()}")



  2%|█▌                                                                              | 1/50 [06:31<5:20:01, 391.86s/it]

Epoch [1/50] | Train Loss: 0.6613 | Validation Loss: 1.4002 | Train acc: 0.4309 | Val acc: 0.1114 | f1 score: 0.1114


  4%|███▏                                                                            | 2/50 [13:02<5:12:58, 391.22s/it]

Epoch [2/50] | Train Loss: 0.3881 | Validation Loss: 1.2071 | Train acc: 0.5012 | Val acc: 0.2835 | f1 score: 0.2835


  6%|████▊                                                                           | 3/50 [19:35<5:06:57, 391.86s/it]

...

Epoch [48/50] | Train Loss: 0.0688 | Validation Loss: 1.4591 | Train acc: 0.8698 | Val acc: 0.5904 | f1 score: 0.5904


 98%|█████████████████████████████████████████████████████████████████████████████▍ | 49/50 [5:20:54<06:33, 393.45s/it]

Epoch [49/50] | Train Loss: 0.0713 | Validation Loss: 1.4458 | Train acc: 0.8649 | Val acc: 0.5846 | f1 score: 0.5846


100%|███████████████████████████████████████████████████████████████████████████████| 50/50 [5:27:26<00:00, 392.93s/it]

Epoch [50/50] | Train Loss: 0.0699 | Validation Loss: 1.4604 | Train acc: 0.8630 | Val acc: 0.5846 | f1 score: 0.5846


In [28]:
print(outputs)

tensor([[ 7.3150, -4.5752,  6.8134, -1.4315],
        [-0.5778,  5.2611, -0.1853, -5.5692],
        [ 6.0844, -0.5860,  3.4989, -3.1107],
        [ 6.9042, -3.0260,  5.4810, -2.4056],
        [ 5.4913, -0.8366,  5.3401, -3.1009],
        [ 7.0985, -3.3910,  4.9623, -3.0410],
        [ 7.3649, -2.9151,  4.9623, -0.9886],
        [ 7.4181, -2.1530,  5.8066, -2.9149],
        [ 5.0642, -1.6110,  5.6315, -4.2222],
        [ 1.9198,  3.3984,  4.4088, -5.2286],
        [ 4.4639, -3.8782,  7.6935, -3.2776],
        [ 5.1656, -2.1504,  6.8736,  0.2348],
        [ 2.4401, -0.5813,  1.3699,  6.3070],
        [ 4.3097, -2.7205,  7.7122, -4.0853],
        [ 0.9133,  1.4447,  5.4773, -0.0272],
        [ 6.7607, -1.6373,  4.3717, -3.9347],
        [ 2.7647,  0.9517,  4.3947, -3.4192],
        [ 7.3140, -3.5348,  6.2206, -2.6685],
        [ 7.2787, -3.4936,  4.2624, -2.4108],
        [ 3.7639,  2.0533,  4.6435, -4.7214],
        [ 3.2696,  1.9262,  3.7876, -3.9049],
        [ 6.5490, -2.6399,  5.1341

In [ ]:
type(labels)

In [ ]:
unique, counts = np.unique(labels, return_counts=True)

In [26]:
dict(zip(unique, counts))

{0: 3030, 1: 443, 2: 1474, 3: 170}

### Test

In [43]:
test_x = pd.read_csv('original_data/test.csv', index_col='id')

filtered_signal_test = []
beats_test = []
for i in range(len(test_x)):
    output = ecg.ecg(test_x.loc[i].dropna().to_numpy(dtype='float32'), sampling_rate=300, show=False)
    filtered = output['filtered']
    beat = output['templates']
    filtered_signal_test.append(filtered)
    beats_test.append(beat)
    if len(filtered) < 1:
        print('filtered {} length is less than one'.format(i))
    if len(beat) < 1:
        print('Beat {} length is less than one'.format(i))
        
with open('data/filtered_ecg_test.pkl', 'wb') as f:
    pickle.dump(filtered_signal_test, f)
    
with open('data/heartbeat_templates_ecg_test.pkl', 'wb') as f:
    pickle.dump(beats_test, f)

In [24]:
with open('data/heartbeat_templates_ecg_test.pkl', 'rb') as f:
    heartbeats = pickle.load(f)

In [25]:
len(heartbeats)

3411

In [26]:
max_length_heartbeats = 159
beat_length = 180 # All heartbeats have the same length

# Padded heartbeats
padded_heartbeats = []
lengths = []

# Should I change the normalizing strategy here??? It is not learned from train data
for i,heartbeat in enumerate(heartbeats):
#     if heartbeat.shape[0] > 159:
#         heartbeat = heartbeat[:159, :]
    heartbeats[i] = (heartbeat - heartbeat.mean(axis=-1, keepdims=True)) / heartbeat.std(axis=-1, keepdims=True)

count = 0
for heartbeat in heartbeats:
    length = len(heartbeat)
    lengths.append(length)
    pad = np.zeros((max_length_heartbeats-length, 180))
    padded_heartbeats.append(np.concatenate((heartbeat, pad),axis=0))

In [27]:
len(padded_heartbeats)

3411

In [28]:
padded_heartbeats = np.array(padded_heartbeats)
padded_heartbeats = torch.tensor(padded_heartbeats, dtype=torch.float32)
labels_heartbeats = torch.tensor(labels)
lengths_heartbeats = torch.tensor(lengths)

In [29]:
test_dataset = SignalDataset(padded_heartbeats, lengths_heartbeats, lengths_heartbeats)

In [30]:
test_dataloader = DataLoader(test_dataset, batch_size=BATCH_SIZE, collate_fn=collate_fn)

In [31]:
model.eval()
all_preds = []
with torch.no_grad():
    for batch in test_dataloader:
        inputs, attention_mask, targets = batch
        inputs = inputs.to(device)
        attention_mask = attention_mask.to(device)
        targets = targets.to(device)
        outputs = model.encode(inputs, attention_mask)
        pred = torch.argmax(outputs, dim=1)
        all_preds.extend(pred.detach().cpu().tolist())
        
# Convert predictions to a numpy array
predictions = np.array(all_preds)

print("Predictions:", predictions)
# length should be 3411
print(len(predictions))

Predictions: [0 2 0 ... 2 0 1]
3411


In [32]:
# generate IDs for each signal in the test set
ids = np.arange(len(predictions))

# Create a DataFrame for submission
submission_df = pd.DataFrame({
    'id': ids,
    'y': predictions
})

# Save the DataFrame to a CSV file
submission_df.to_csv('./data/predictions/test_preds2.csv', index=False)